In [1]:
from run_bench import save_overall_results
from pathlib import Path

/Users/user/demo_1/oobabooga_macos/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fig = save_overall_results(results_folder=Path("results"), method="plotly")

100%|██████████| 9/9 [00:00<00:00, 28684.45it/s]


Loading datasets with seed: 1
cais_mmlu-all-validation 0.25
ai2_arc-arc-challenge-validation 0.24968789013732834
anli--test_r1 0.3333333333333333
cosmos_qa--validation 0.25
alekseykorshuk_hellaswag-arc-easy-validation 0.25
winogrande-winogrande_s-validation 0.5
race-high-validation 0.25
math_qa--validation 0.2004008016032064
truthful_qa-multiple_choice-validation 0.19880715705765406
0.029834985733032227


100%|██████████| 9/9 [00:00<00:00, 100129.27it/s]


Loading datasets with seed: 1
cais_mmlu-all-validation 0.25
ai2_arc-arc-challenge-validation 0.24968789013732834
anli--test_r1 0.3333333333333333
cosmos_qa--validation 0.25
alekseykorshuk_hellaswag-arc-easy-validation 0.25
winogrande-winogrande_s-validation 0.5
race-high-validation 0.25
math_qa--validation 0.2004008016032064
truthful_qa-multiple_choice-validation 0.19880715705765406
0.00824880599975586


100%|██████████| 9/9 [00:00<00:00, 113701.01it/s]


...

Loading datasets with seed: 1
cais_mmlu-all-validation 0.25
ai2_arc-arc-challenge-validation 0.24968789013732834
anli--test_r1 0.3333333333333333
cosmos_qa--validation 0.25
alekseykorshuk_hellaswag-arc-easy-validation 0.25
winogrande-winogrande_s-validation 0.5
race-high-validation 0.25
math_qa--validation 0.2004008016032064
truthful_qa-multiple_choice-validation 0.19880715705765406
0.011543035507202148


100%|██████████| 9/9 [00:00<00:00, 91846.07it/s]


Loading datasets with seed: 1
cais_mmlu-all-validation 0.25
ai2_arc-arc-challenge-validation 0.24968789013732834
anli--test_r1 0.3333333333333333
cosmos_qa--validation 0.25
alekseykorshuk_hellaswag-arc-easy-validation 0.25
winogrande-winogrande_s-validation 0.5
race-high-validation 0.25
math_qa--validation 0.2004008016032064
truthful_qa-multiple_choice-validation 0.19880715705765406
0.011056900024414062


100%|██████████| 9/9 [00:00<00:00, 19909.67it/s]


Loading datasets with seed: 1
cais_mmlu-all-validation 0.25
ai2_arc-arc-challenge-validation 0.24968789013732834
anli--test_r1 0.3333333333333333
cosmos_qa--validation 0.25
alekseykorshuk_hellaswag-arc-easy-validation 0.25
winogrande-winogrande_s-validation 0.5
race-high-validation 0.25
math_qa--validation 0.2004008016032064
truthful_qa-multiple_choice-validation 0.19880715705765406
0.011353254318237305
